In [2]:
import getpass
import os
from IPython.display import JSON

from langchain.chat_models import init_chat_model

llm = init_chat_model("gemini-2.0-flash")
output = llm.invoke("Tell me a joke about cats")


GoogleAuthError: Unable to find your project. Please provide a project ID by:
- Passing a constructor argument
- Using vertexai.init()
- Setting project using 'gcloud config set project my-project'
- Setting a GCP environment variable
- To create a Google Cloud project, please follow guidance at https://developers.google.com/workspace/guides/create-project

In [7]:
from pygments import highlight
from pygments.lexers import JsonLexer
from pygments.formatters import HtmlFormatter
from IPython.display import HTML
import json

def display_json_with_highlight(output):
    # pprint.pprint(output.to_json(), indent=1)
    formatted_json = json.dumps(output.to_json(), indent=2)
    formatter = HtmlFormatter(style='dracula', noclasses=True) 
    highlighted_html = highlight(formatted_json, JsonLexer(), formatter)
    pygments_style_defs = formatter.get_style_defs('.highlight')
    custom_css = f"""
    <style>
        /* Pygments Dracula 主题的 CSS 规则 */
        {pygments_style_defs}

        /* 覆盖 Pygments 默认字体，并确保使用 Fira Code */
        .highlight, .highlight pre, .highlight code {{
            font-family: "Fira Code", monospace !important;
            font-size: 2 em !important; /* 调整字体大小 */
            line-height: 1.5 !important; /* 调整行高 */
        }}

        /* 确保整个高亮块的背景色和文本颜色是 Dracula 的，并提供一些内边距 */
        /* .output_area 是 Jupyter 输出区域的类 */
        .output_area pre.highlight {{
            background-color: #282a36 !important; /* Dracula 的背景色 */
            color: #f8f8f2 !important; /* Dracula 的默认文本色 */
            padding: 1em !important;
            border-radius: 4px !important;
            overflow-x: auto !important; /* 防止内容溢出，添加滚动条 */
        }}

        /* 对于代码内部的特定 Pygments 颜色类，如果需要微调，可以在这里添加 */
        /* 例如：
        .highlight .k {{ color: #ff79c6 !important; }} /* Dracula 的关键字颜色 */
        */
    </style>
    """
    return highlighted_html+custom_css

In [5]:
ds_llm = init_chat_model(
    "deepseek/deepseek-chat-v3-0324:free", 
    model_provider="openai",
    base_url=os.getenv("OPENROUTER_BASE_URL"), 
    openai_api_key=os.getenv("OPENROUTER_API_KEY"),
)
output = ds_llm.invoke("Tell me a joke about cats")
display_json_with_highlight(output)

In [8]:
HTML(display_json_with_highlight(output))

In [ ]:
from typing import Optional
from pydantic import BaseModel, Field
# Pydantic
class Joke(BaseModel):
    """Joke to tell user."""
    setup: str = Field(description="The setup of the joke")
    punchline: str = Field(description="The punchline to the joke")
    rating: Optional[int] = Field(
        default=None, description="How funny the joke is, from 1 to 10"
    )

structured_llm = llm.with_structured_output(Joke)
structured_llm.invoke("Tell me a joke about cats")